In [ ]:
#installing psycopg2
pip install psycopg2

In [ ]:
#Installing google-cloud-storage
pip install google-cloud-storage

In [ ]:
#Installing BigQuery
pip install google-cloud-bigquery

In [4]:
#Installing sqlalchemy
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [5]:
#importing necessary libraries
import psycopg2
import pandas as pd
import os
from google.cloud import storage, bigquery
from sqlalchemy import create_engine

In [6]:
# setting the environment variable to the path of a JSON file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "E:/DE project/AIQuest/etl-bde3-8127e3bdf05d.json"


In [7]:
#creating instance for google cloud storage
storage_client = storage.Client()

In [8]:
#creating instance for bigquery client
bigquery_client = bigquery.Client()

In [9]:
#establishing connection to postgesql 
rds_conn = psycopg2.connect(
    host="bde3-project.cqcqebjkmcrf.us-east-1.rds.amazonaws.com",
    database="abdur",
    user="abdur",
    password="*******"
)

In [10]:
#define sql queries to select all columns from tables
sql_query_Customers = "SELECT * FROM Customers"
sql_query_Orders = "SELECT * FROM Orders"
sql_query_OrderDetails = "SELECT * FROM OrderDetails"
#reading sql queries
df_Products = pd.read_sql(sql_query_Products, rds_conn)
df_Customers = pd.read_sql(sql_query_Customers, rds_conn)
df_Orders = pd.read_sql(sql_query_Orders, rds_conn)
df_OrderDetails = pd.read_sql(sql_query_OrderDetails, rds_conn)

C:\Users\Abdur\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\Abdur\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\Abdur\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\Abdur\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqli

In [11]:
#Exporting Dataframe to CSV
df_Products.to_csv('Products.csv', index=False)
df_Customers.to_csv('Customers.csv', index=False)
df_Orders.to_csv('Orders.csv', index=False)
df_OrderDetails.to_csv('OrderDetails.csv', index=False)

In [12]:
project_id = 'etl-bde3'
bucket_name = 'bde3-de-project'
files_to_upload = ['Products.csv', 'Customers.csv', 'Orders.csv', 'OrderDetails.csv']
storage_client = storage.Client(project=project_id)

# Iterate over each file and upload to GCS
for file_name in files_to_upload:
    blob_name = file_name
    
    # Create a blob object
    blob = storage_client.bucket(bucket_name).blob(blob_name)
    
    # Upload the file to GCS
    blob.upload_from_filename(file_name)

    print(f"File {file_name} uploaded to GCS bucket {bucket_name}.")


File Products.csv uploaded to GCS bucket bde3-de-project.
File Customers.csv uploaded to GCS bucket bde3-de-project.
File Orders.csv uploaded to GCS bucket bde3-de-project.
File OrderDetails.csv uploaded to GCS bucket bde3-de-project.


In [18]:
project_id = 'etl-bde3'
dataset_id = 'etl_bde3_data'
bucket_name = 'bde3-de-project'
files_to_upload = ['Customers.csv', 'Products.csv', 'Orders.csv', 'OrderDetails.csv']

# Create a BigQuery client
bigquery_client = bigquery.Client(project=project_id)

# Check if the dataset exists, create it if not
dataset_ref = bigquery_client.dataset(dataset_id)
try:
    bigquery_client.get_dataset(dataset_ref)
except Exception as e:
    print(f"Dataset '{dataset_id}' not found. Error: {e}")
    print(f"Creating dataset '{dataset_id}'.")
    dataset = bigquery.Dataset(dataset_ref)
    bigquery_client.create_dataset(dataset)

# Load each CSV file into a table in BigQuery
for file_name in files_to_upload:
    # Construct table_id from file_name (remove extension and replace non-alphanumeric characters)
    table_id = f'{file_name.split(".")[0]}'

    table_ref = dataset_ref.table(table_id)

    # Check if the table exists, create it if not
    try:
        bigquery_client.get_table(table_ref)
    except Exception as e:
        print(f"Table '{table_id}' not found. Error: {e}")
        print(f"Creating table '{table_id}' in dataset '{dataset_id}'.")
        
        # Define table schema based on the structure of CSV files
        
        job_config = bigquery.LoadJobConfig()
        job_config.source_format = bigquery.SourceFormat.CSV
        job_config.skip_leading_rows = 1
        job_config.autodetect = True

        # Create an empty table with the specified schema
        table = bigquery.Table(table_ref)
        bigquery_client.create_table(table)

    uri = f"gs://{bucket_name}/{file_name}"
    load_job = bigquery_client.load_table_from_uri(uri, table_ref, job_config=job_config)

    print(f"Starting job {load_job.job_id}")
    load_job.result()  # Wait for table load to complete
    print(f"Job finished. Loaded {load_job.output_rows} rows into {dataset_id}:{table_id}.")


print("Data loading and table creation completed successfully.")

Table 'Customers' not found. Error: 404 GET https://bigquery.googleapis.com/bigquery/v2/projects/etl-bde3/datasets/etl_bde3_data/tables/Customers?prettyPrint=false: Not found: Table etl-bde3:etl_bde3_data.Customers
Creating table 'Customers' in dataset 'etl_bde3_data'.
Starting job 8ccee351-2afb-4239-bf87-e0cccae3d0d2
Job finished. Loaded 20 rows into etl_bde3_data:Customers.
Table 'Products' not found. Error: 404 GET https://bigquery.googleapis.com/bigquery/v2/projects/etl-bde3/datasets/etl_bde3_data/tables/Products?prettyPrint=false: Not found: Table etl-bde3:etl_bde3_data.Products
Creating table 'Products' in dataset 'etl_bde3_data'.
Starting job d49a01c5-e758-4e00-b2a7-f8d2c5e8eb91
Job finished. Loaded 18 rows into etl_bde3_data:Products.
Table 'Orders' not found. Error: 404 GET https://bigquery.googleapis.com/bigquery/v2/projects/etl-bde3/datasets/etl_bde3_data/tables/Orders?prettyPrint=false: Not found: Table etl-bde3:etl_bde3_data.Orders
Creating table 'Orders' in dataset 'etl_b